# LangSmith API Key 로드

In [4]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()

True

# Langsmith로 프로젝트 추적 설정

In [5]:
from utils import logging

logging.langsmith("Prompt")

LangSmith 추적을 시작합니다.
[프로젝트명]
RAG_prompt


# LLM 객체 정의

In [9]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI()

# Prompt

프롬프트의 필요성

1. 문맥(Context) 설정 <br>
<tab> 프롬프트는 언어 모델이 특정 문맥에서 작동하도록 설정하는 역할을 합니다. <br>
<tab> 이를 통해 모델은 제공된 정보를 바탕으로 보다 정확하고 관련성 높은 답변을 생성할 수 있습니다. <br>

2. 정보 통합 <br>
<tab> 여러 문서에서 검색된 정보는 서로 다른 관점이나 내용을 포함할 수 있습니다. <br>
<tab> 프롬프트 단계에서 이러한 정보를 통합하고, 모델이 이를 효율적으로 활용할 수 있는 형식으로 조정합니다.

3. 응답 품질 향상 <br>
<tab> 질문에 대한 모델의 응답 품질은 프롬프트의 구성에 크게 의존합니다. <br>
<tab> 잘 구성된 프롬프트는 모델이 보다 정확하고 유용한 정보를 제공하게 돕습니다.

RAG 프롬프트 구조는 다음과 같습니다.

지시사항(Instruction)
```
당신은 질문-답변(Question-Answer) Task 를 수행한는 AI 어시스턴트 입니다.
검색된 문맥(context)를 사용하여 질문(question)에 답하세요. 
만약, 문맥(context) 으로부터 답을 찾을 수 없다면 '모른다' 고 말하세요. 
한국어로 대답하세요.
```

질문(사용자 입력 질문)
```
Question: 
{이곳에 사용자가 입력한 질문이 삽입됩니다}
```

문맥(검색된 정보)
```
Context: 
{이곳에 검색된 정보가 삽입됩니다}
```

# ChatPromptTemplate

ChatPromptTemplate 은 대화목록을 프롬프트로 주입하고자 할 때 활용할 수 있습니다.

메시지는 튜플(tuple) 형식으로 구성하며, (role, message) 로 구성하여 리스트로 생성할 수 있습니다.

role 

- "system": 시스템 설정 메시지 입니다. 주로 전역설정과 관련된 프롬프트입니다. 

- "human" : 사용자 입력 메시지 입니다. 

- "ai": AI 의 답변 메시지입니다.

In [10]:
from langchain_core.prompts import ChatPromptTemplate

chat_template = ChatPromptTemplate.from_messages(
    [
        # role, message
        ("system", "당신은 친절한 AI 어시스턴트입니다. 당신의 이름은 {name} 입니다."),
        ("human", "반가워요!"),
        ("ai", "안녕하세요! 무엇을 도와드릴까요?"),
        ("human", "{user_input}"),
    ]
)

# 챗 message 를 생성합니다.
messages = chat_template.format_messages(
    name="앵무새", user_input="당신의 이름은 무엇입니까?"
)
messages

[SystemMessage(content='당신은 친절한 AI 어시스턴트입니다. 당신의 이름은 앵무새 입니다.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='반가워요!', additional_kwargs={}, response_metadata={}),
 AIMessage(content='안녕하세요! 무엇을 도와드릴까요?', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='당신의 이름은 무엇입니까?', additional_kwargs={}, response_metadata={})]

생성한 메시지를 바로 주입하여 결과를 받을 수 있습니다.

In [11]:
llm.invoke(messages).content

'제 이름은 앵무새입니다. 어떻게 도와드릴까요?'

이번에는 체인을 생성해 보겠습니다.

In [13]:
chain = chat_template | llm

chain.invoke({"name": "앵무새", "user_input": "당신의 이름은 무엇입니까?"}).content

'제 이름은 앵무새입니다. 어떻게 도와드릴까요?'

# MessagePlaceholder

 LangChain은 포맷하는 동안 렌더링할 메시지를 완전히 제어할 수 있는 MessagePlaceholder 를 제공합니다.

메시지 프롬프트 템플릿에 어떤 역할을 사용해야 할지 확실하지 않거나 서식 지정 중에 메시지 목록을 삽입하려는 경우 유용할 수 있습니다.

In [30]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

chat_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "당신은 요약 전문 AI 어시스턴트입니다. 당신의 임무는 주요 키워드로 대화를 요약하는 것입니다.",
        ),
        MessagesPlaceholder(variable_name="conversation"),
        ("system", "지금까지의 내용을 {word_count} 한 문장으로 요약합니다."),
    ]
)
chat_prompt

ChatPromptTemplate(input_variables=['conversation', 'word_count'], input_types={'conversation': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='ChatMessageChunk')], typing.Annotated[langchain_core.messages.system.SystemMessageChunk, Tag(tag='SystemMessageChunk')], typing.Annota

conversation 대화목록을 나중에 추가하고자 할 때 MessagesPlaceholder 를 사용할 수 있습니다.

In [31]:
# formatted_chat_prompt = chat_prompt.format(
#     word_count=5,
#     conversation = [
#         ("human", "제가 드린 보고서 내용 확인해주실 수 있나요? 추가로 요약이 필요해요."),
#         ("ai", "네, 보고서 확인했습니다. 어떤 부분을 요약해드릴까요?"),
#         ("human", "경제 전망 부분을 중점으로 요약해주시고, 최신 데이터를 반영해주시면 좋을 것 같아요."),
#         ("ai", "알겠습니다. 경제 전망 부분을 요약하고 최신 데이터를 찾아서 반영하겠습니다."),
#         ("human", "추가로, 인플레이션에 대한 부분도 간단히 요약해주시겠어요?"),
#         ("ai", "네, 인플레이션에 대한 내용도 함께 요약해서 보내드리겠습니다."),
#         ("human", "감사합니다! 요약 완료되면 바로 공유 부탁드려요."),
#         ("ai", "좋습니다. 요약이 완료되면 바로 공유드리겠습니다."),
#     ],
# )

# print(formatted_chat_prompt)

System: 당신은 요약 전문 AI 어시스턴트입니다. 당신의 임무는 주요 키워드로 대화를 요약하는 것입니다.
Human: 제가 드린 보고서 내용 확인해주실 수 있나요? 추가로 요약이 필요해요.
AI: 네, 보고서 확인했습니다. 어떤 부분을 요약해드릴까요?
Human: 경제 전망 부분을 중점으로 요약해주시고, 최신 데이터를 반영해주시면 좋을 것 같아요.
AI: 알겠습니다. 경제 전망 부분을 요약하고 최신 데이터를 찾아서 반영하겠습니다.
Human: 추가로, 인플레이션에 대한 부분도 간단히 요약해주시겠어요?
AI: 네, 인플레이션에 대한 내용도 함께 요약해서 보내드리겠습니다.
Human: 감사합니다! 요약 완료되면 바로 공유 부탁드려요.
AI: 좋습니다. 요약이 완료되면 바로 공유드리겠습니다.
System: 지금까지의 내용을 5 한 문장으로 요약합니다.


In [33]:
# chain 생성
chain = chat_prompt | llm | StrOutputParser()

# chain 실행 및 결과확인
chain.invoke(
    {
        "word_count": 5,
        "conversation" : [
            ("human", "제가 드린 보고서 내용 확인해주실 수 있나요? 추가로 요약이 필요해요."),
            ("ai", "네, 보고서 확인했습니다. 어떤 부분을 요약해드릴까요?"),
            ("human", "경제 전망 부분을 중점으로 요약해주시고, 최신 데이터를 반영해주시면 좋을 것 같아요."),
            ("ai", "알겠습니다. 경제 전망 부분을 요약하고 최신 데이터를 찾아서 반영하겠습니다."),
            ("human", "추가로, 인플레이션에 대한 부분도 간단히 요약해주시겠어요?"),
            ("ai", "네, 인플레이션에 대한 내용도 함께 요약해서 보내드리겠습니다."),
            ("human", "감사합니다! 요약 완료되면 바로 공유 부탁드려요."),
            ("ai", "좋습니다. 요약이 완료되면 바로 공유드리겠습니다."),
        ],
    }
)


'요청하신 보고서의 경제 전망과 인플레이션 부분을 요약해드릴 예정이며, 최신 데이터를 참고하여 내용을 반영하겠습니다. 완료되면 즉시 공유할 예정입니다.'

# Langchain Hub

다음은 LangChain Hub 에서 프롬프트를 받아서 실행하는 예제입니다.

아래 주소에서 LangChain Hub 프롬프트를 확인할 수 있습니다.

받아오는 방법은 프롬프트 repo 의 아이디 값을 가져 올 수 있고, commit id 를 붙여서 특정 버전에 대한 프롬프트를 받아올 수도 있습니다.

## Hub로부터 Prompt 받아오기

In [34]:
from langchain import hub

# 가장 최신 버전의 프롬프트를 가져옵니다.
prompt = hub.pull("rlm/rag-prompt")

In [35]:
# 프롬프트 내용 출력
print(prompt)

input_variables=['context', 'question'] input_types={} partial_variables={} metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})]


In [36]:
# 특정 버전의 프롬프트를 가져오려면 버전 해시를 지정하세요
prompt = hub.pull("rlm/rag-prompt:50442af1")
prompt

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})])

## Prompt Hub 에 자신의 프롬프트 등록

In [ ]:
from langchain.prompts import ChatPromptTemplate


prompt = ChatPromptTemplate.from_template(
    "주어진 내용을 바탕으로 다음 문장을 요약하세요. 답변은 반드시 한글로 작성하세요\n\nCONTEXT: {context}\n\nSUMMARY:"
)


from langchain import hub

# 프롬프트를 허브에 업로드합니다.
hub.push("selena/simple-summary-korean", prompt)



In [49]:
from langchain.prompts import ChatPromptTemplate


prompt = ChatPromptTemplate.from_template(
    "주어진 내용을 바탕으로 다음 문장을 요약하세요. 답변은 반드시 한글로 작성하세요\n\nCONTEXT: {context}\n\nSUMMARY:"
)
prompt


ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='주어진 내용을 바탕으로 다음 문장을 요약하세요. 답변은 반드시 한글로 작성하세요\n\nCONTEXT: {context}\n\nSUMMARY:'), additional_kwargs={})])

Prompt 최초 생성 시, 아래 코드 실행해서 기본 설정인 private을 public으로 변경 후 langsmith 웹사이트에서 owner 이름을 지정할 수 있음

In [51]:
from langchain import hub

# 프롬프트를 허브에 업로드합니다.
hub.push("simple-summary-korean", prompt)


'https://smith.langchain.com/prompts/simple-summary-korean/de93adb6?organizationId=abe0fe3c-f07a-458f-b5f6-27c6aefdb9ed'

웹사이트에서 public으로 설정하면 owner 이름을 아래와 같이 'selena'로 지정할 수 있음

그 후에 아래 코드를 실행하면 프롬프트 내용 출력이 가능함

In [52]:
from langchain import hub

# 프롬프트를 허브로부터 가져옵니다.
pulled_prompt = hub.pull("selena/simple-summary-korean")


In [53]:
# 프롬프트 내용 출력
print(pulled_prompt)


input_variables=['context'] input_types={} partial_variables={} metadata={'lc_hub_owner': 'selena', 'lc_hub_repo': 'simple-summary-korean', 'lc_hub_commit_hash': 'de93adb6bb86de943383a02bd60acdcaa6a5d6f7876d2901439bbb30a5e90918'} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='주어진 내용을 바탕으로 다음 문장을 요약하세요. 답변은 반드시 한글로 작성하세요\n\nCONTEXT: {context}\n\nSUMMARY:'), additional_kwargs={})]


## 개인화된 프롬프트(Hub에 업로드)

LangChain 아이디를 입력합니다.

In [54]:
# Owner 지정
PROMPT_OWNER = "selena"

In [55]:
from langchain import hub
from langchain.prompts import PromptTemplate

prompt_title = "summary-stuff-documents"

# 요약문을 작성하기 위한 프롬프트 정의 (직접 프롬프트를 작성하는 경우)
prompt_template = """Please summarize the sentence according to the following REQUEST.
REQUEST:
1. Summarize the main points in bullet points.
2. Each summarized sentence must start with an emoji that fits the meaning of the each sentence.
3. Use various emojis to make the summary more interesting.
4. DO NOT include any unnecessary information.

CONTEXT:
{context}

SUMMARY:"
"""
prompt = PromptTemplate.from_template(prompt_template)
prompt


PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='Please summarize the sentence according to the following REQUEST.\nREQUEST:\n1. Summarize the main points in bullet points.\n2. Each summarized sentence must start with an emoji that fits the meaning of the each sentence.\n3. Use various emojis to make the summary more interesting.\n4. DO NOT include any unnecessary information.\n\nCONTEXT:\n{context}\n\nSUMMARY:"\n')

In [56]:
hub.push(f"{PROMPT_OWNER}/{prompt_title}", prompt)

'https://smith.langchain.com/prompts/summary-stuff-documents/ccf1dc3e?organizationId=abe0fe3c-f07a-458f-b5f6-27c6aefdb9ed'

## RAG 문서 프롬프트

In [57]:
prompt_title = "rag-prompt-korean"

In [58]:
system = """
당신은 질문-답변(Question-Answering)을 수행하는 친절한 AI 어시스턴트입니다. 
당신의 임무는 주어진 문맥(context) 에서 주어진 질문(question) 에 답하는 것입니다.
검색된 다음 문맥(context) 을 사용하여 질문(question) 에 답하세요. 
만약, 주어진 문맥(context) 에서 답을 찾을 수 없다면, 답을 모른다면 `주어진 정보에서 질문에 대한 정보를 찾을 수 없습니다` 라고 답하세요.
한글로 답변해 주세요. 
단, 기술적인 용어나 이름은 번역하지 않고 그대로 사용해 주세요. 
Don't narrate the answer, just answer the question. Let's think step-by-step.
"""

human = """
#Question: 
{question} 

#Context: 
{context} 

#Answer:"""

from langchain.prompts import ChatPromptTemplate


prompt = ChatPromptTemplate.from_messages([("system", system), ("human", human)])


In [59]:
hub.push(f"{PROMPT_OWNER}/{prompt_title}", prompt, parent_commit_hash="latest")

'https://smith.langchain.com/prompts/rag-prompt-korean/3ae976de?organizationId=abe0fe3c-f07a-458f-b5f6-27c6aefdb9ed'